# 티어 별 분석

In [ ]:
import pandas as pd
import os
from collections import defaultdict

# ===============================
# 0. 설정
# ===============================
BASE_PATH = "../../data/processed"

TIERS = [
    "IRON_I", "IRON_II", "IRON_III", "IRON_IV",
    "BRONZE_I", "BRONZE_II", "BRONZE_III", "BRONZE_IV",
    "SILVER_I", "SILVER_II", "SILVER_III", "SILVER_IV",
    "GOLD_I", "GOLD_II", "GOLD_III", "GOLD_IV",
    "PLATINUM_I", "PLATINUM_II", "PLATINUM_III", "PLATINUM_IV",
    "EMERALD_I", "EMERALD_II", "EMERALD_III", "EMERALD_IV",
    "DIAMOND_I", "DIAMOND_II", "DIAMOND_III", "DIAMOND_IV",
    "MASTER", "GRANDMASTER", "CHALLENGER"
]

# 🔥 goldDiff 포함
METRICS = {
    "goldDiff": 500,          # 골드는 500 단위
    "totalKillDiff": 1,
    "dragonDiff": 1,
    "elderDiff": 1,
    "heraldDiff": 1,
    "baronDiff": 1,
    "atakhanDiff": 1,
    "grubDiff": 1,
    "outerTowerDiff": 1,
    "innerTowerDiff": 1,
    "baseTowerDiff": 1,
}

MIN_SAMPLES = 50


# ===============================
# 1. 대분류 티어 변환
# ===============================
def get_major_tier(tier_name: str) -> str:
    if "_" in tier_name:
        return tier_name.split("_")[0]
    return tier_name


# ===============================
# 2. diff → 승률 분석 함수
# ===============================
def analyze_metric(df, tier, metric, bin_size):
    temp = df.copy()
    temp["diff_bin"] = (temp[metric] // bin_size) * bin_size

    res = (
        temp
        .groupby("diff_bin")["win"]
        .agg(["mean", "count"])
        .reset_index()
        .rename(columns={
            "mean": "win_rate",
            "count": "num_samples"
        })
    )

    res = res[res["num_samples"] >= MIN_SAMPLES]

    res["tier"] = tier
    res["metric"] = metric

    return res[["tier", "metric", "diff_bin", "win_rate", "num_samples"]]


# ===============================
# 3. 대분류 티어별 long_df 누적
# ===============================
tier_long_dfs = defaultdict(list)

for tier in TIERS:
    timeline_path = f"{BASE_PATH}/{tier}_timeline.csv"
    matches_path  = f"{BASE_PATH}/{tier}_matches.csv"

    if not os.path.exists(timeline_path):
        print(f"[SKIP] {tier} timeline 없음")
        continue

    print(f"[LOAD] {tier}")

    timeline = pd.read_csv(timeline_path)
    matches  = pd.read_csv(matches_path)

    winner_team = (
        matches[matches["win"] == 1]
        .rename(columns={"teamId": "winnerTeamId"})[["matchId", "winnerTeamId"]]
    )

    df = timeline.merge(winner_team, on="matchId", how="left")
    df["win"] = (df["winnerTeamId"] == 100).astype(int)

    rows = []
    for _, row in df.iterrows():
        max_min = int(row["maxMinute"])
        for minute in range(15, max_min + 1, 5):
            suffix = f"_{minute}"
            if f"goldDiff{suffix}" not in row:
                continue

            rows.append({
                "win": row["win"],
                "goldDiff": row[f"goldDiff{suffix}"],

                "totalKillDiff": row[f"totalKillDiff{suffix}"],
                "dragonDiff": row[f"dragonDiff{suffix}"],
                "elderDiff": row[f"elderDiff{suffix}"],
                "heraldDiff": row[f"heraldDiff{suffix}"],
                "baronDiff": row[f"baronDiff{suffix}"],
                "atakhanDiff": row[f"atakhanDiff{suffix}"],
                "grubDiff": row[f"grubDiff{suffix}"],

                # 구조물은 방향 보정
                "outerTowerDiff": -row[f"outerTowerDiff{suffix}"],
                "innerTowerDiff": -row[f"innerTowerDiff{suffix}"],
                "baseTowerDiff":  -row[f"baseTowerDiff{suffix}"],
            })

    major_tier = get_major_tier(tier)
    tier_long_dfs[major_tier].append(pd.DataFrame(rows))


# ===============================
# 4. 대분류 티어 단위로 분석
# ===============================
all_results = []

for major_tier, df_list in tier_long_dfs.items():
    tier_df = pd.concat(df_list, ignore_index=True)

    for metric, bin_size in METRICS.items():
        result = analyze_metric(tier_df, major_tier, metric, bin_size)
        all_results.append(result)


# ===============================
# 5. 최종 결과
# ===============================
final_df = pd.concat(all_results, ignore_index=True)

print("\n=== 전 티어(대분류) × 지표 × diff_bin 승률 비교 ===")
display(final_df.sort_values(["metric", "tier", "diff_bin"]))


[LOAD] IRON_I
[LOAD] IRON_II
[LOAD] IRON_III
[LOAD] IRON_IV
[LOAD] BRONZE_I
[LOAD] BRONZE_II
[LOAD] BRONZE_III
[LOAD] BRONZE_IV
[LOAD] SILVER_I
[LOAD] SILVER_II
[LOAD] SILVER_III
[LOAD] SILVER_IV
[LOAD] GOLD_I
[LOAD] GOLD_II
[LOAD] GOLD_III
[LOAD] GOLD_IV
[LOAD] PLATINUM_I
[LOAD] PLATINUM_II
[LOAD] PLATINUM_III
[LOAD] PLATINUM_IV
[LOAD] EMERALD_I
[LOAD] EMERALD_II
[LOAD] EMERALD_III
[LOAD] EMERALD_IV
[LOAD] DIAMOND_I
[LOAD] DIAMOND_II
[LOAD] DIAMOND_III
[LOAD] DIAMOND_IV
[LOAD] MASTER
[LOAD] GRANDMASTER
[LOAD] CHALLENGER

=== 전 티어(대분류) × 지표 × diff_bin 승률 비교 ===


,tier,metric,diff_bin,win_rate,num_samples
300,BRONZE,atakhanDiff,-1.0,0.295603,16035
301,BRONZE,atakhanDiff,0.0,0.521245,49305
302,BRONZE,atakhanDiff,1.0,0.746264,15390
1572,CHALLENGER,atakhanDiff,-1.0,0.309917,4840
1573,CHALLENGER,atakhanDiff,0.0,0.528595,23780
...,...,...,...,...,...
441,SILVER,totalKillDiff,22.0,1.000000,155
442,SILVER,totalKillDiff,23.0,0.933333,150
443,SILVER,totalKillDiff,24.0,1.000000,135
444,SILVER,totalKillDiff,25.0,1.000000,110


# 결과 파일 생성

In [17]:
import os

# ===============================
# 저장 디렉터리 생성
# ===============================
BASE_DIR = "../../data/analysis/tier"
os.makedirs(BASE_DIR, exist_ok=True)

# ===============================
# CSV 저장
# ===============================
final_df.to_csv(
    os.path.join(BASE_DIR, "tier_metric_winrate.csv"),
    index=False,
    encoding="utf-8-sig"
)

print("저장 완료: data/analysis/tier/tier_metric_winrate.csv")


저장 완료: data/analysis/tier/tier_metric_winrate.csv


# 티어 차이 분석

In [36]:
import pandas as pd

# ==================================================
# 1. CSV 로드
# ==================================================
final_df = pd.read_csv(
    "../../data/analysis/tier/tier_metric_winrate.csv",
    encoding="utf-8-sig"
)

# ==================================================
# 2. 티어 순서 정의
# ==================================================
TIER_ORDER = [
    "IRON", "BRONZE", "SILVER", "GOLD",
    "PLATINUM", "EMERALD", "DIAMOND",
    "MASTER", "GRANDMASTER", "CHALLENGER"
]

final_df["tier"] = pd.Categorical(
    final_df["tier"],
    categories=TIER_ORDER,
    ordered=True
)

# ==================================================
# 3. grubDiff 범위 강제 제한 (-3 ~ +3)
# ==================================================
final_df = final_df[
    ~(
        (final_df["metric"] == "grubDiff") &
        ((final_df["diff_bin"] < -3) | (final_df["diff_bin"] > 3))
    )
]

# ==================================================
# 4. metric 목록
# ==================================================
metrics = final_df["metric"].unique()

# ==================================================
# 5. metric별 Pivot 테이블 생성
# ==================================================
pivot_tables = {}

for metric in metrics:
    metric_df = final_df[final_df["metric"] == metric].copy()

    if metric_df.empty:
        continue

    pivot = (
        metric_df
        .pivot(
            index="tier",        # 행: 티어
            columns="diff_bin",  # 열: diff 값
            values="win_rate"    # 값: 승률
        )
        .sort_index()           # 티어 순서
        .sort_index(axis=1)     # diff_bin 오름차순
    )

    pivot_tables[metric] = pivot

# ==================================================
# 6. 출력
# ==================================================
for metric, table in pivot_tables.items():
    print(f"\n=== Tier × {metric} 승률 비교 ===")
    display(table.round(3))



=== Tier × goldDiff 승률 비교 ===


diff_bin,-16000.0,-15000.0,-14500.0,-14000.0,-13500.0,-13000.0,-12500.0,-12000.0,-11500.0,-11000.0,...,10000.0,10500.0,11000.0,11500.0,12000.0,12500.0,13000.0,13500.0,14000.0,14500.0
tier,,,,,,,,,,,,,,,,,,,,,
IRON,0.0,NaN,NaN,0.000,0.000,0.000,0.000,0.025,0.000,0.058,...,0.978,1.000,0.978,1.000,0.955,1.000,1.0,1.0,1.0,NaN
BRONZE,NaN,0.0,0.0,0.000,0.000,0.000,0.042,0.024,0.038,0.000,...,1.000,0.947,1.000,0.980,0.950,1.000,1.0,1.0,NaN,NaN
SILVER,NaN,NaN,0.0,0.000,0.000,0.000,0.000,0.000,0.020,0.016,...,0.978,0.987,1.000,1.000,1.000,0.962,1.0,1.0,1.0,NaN
GOLD,NaN,0.0,0.0,0.062,0.000,0.077,0.000,0.000,0.023,0.000,...,0.988,0.985,1.000,1.000,1.000,1.000,1.0,1.0,1.0,1.0
PLATINUM,NaN,0.0,0.0,0.000,0.056,0.056,0.000,0.026,0.000,0.014,...,1.000,1.000,1.000,0.977,1.000,1.000,1.0,1.0,1.0,NaN
EMERALD,NaN,NaN,0.0,0.062,0.000,0.038,0.053,0.000,0.017,0.034,...,0.986,1.000,0.977,0.973,1.000,0.957,1.0,NaN,1.0,NaN
DIAMOND,NaN,NaN,0.0,0.000,0.000,0.000,0.000,0.000,0.023,0.000,...,0.984,1.000,0.956,0.971,0.964,1.000,1.0,NaN,NaN,NaN
MASTER,NaN,NaN,0.0,0.000,0.000,0.000,0.031,0.000,0.029,0.000,...,0.973,1.000,1.000,1.000,0.960,1.000,1.0,1.0,NaN,NaN
GRANDMASTER,NaN,NaN,NaN,NaN,0.000,NaN,0.053,0.000,0.000,0.000,...,1.000,0.968,1.000,0.969,1.000,1.000,NaN,1.0,NaN,NaN



=== Tier × totalKillDiff 승률 비교 ===


diff_bin,-27.0,-26.0,-25.0,-24.0,-23.0,-22.0,-21.0,-20.0,-19.0,-18.0,...,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0
tier,,,,,,,,,,,,,,,,,,,,,
IRON,0.091,0.0,0.083,0.111,0.156,0.119,0.055,0.099,0.065,0.061,...,0.969,0.965,0.922,0.982,0.974,0.977,1.000,0.957,0.920,1.0
BRONZE,0.000,NaN,0.053,0.048,0.000,0.023,0.022,0.038,0.054,0.042,...,0.957,0.936,0.957,0.960,0.844,0.972,0.955,1.000,0.882,NaN
SILVER,0.100,NaN,0.050,0.000,0.000,0.000,0.036,0.035,0.083,0.067,...,0.897,0.951,1.000,0.981,1.000,0.933,1.000,1.000,1.000,NaN
GOLD,NaN,NaN,NaN,0.000,0.000,0.037,0.000,0.056,0.048,0.039,...,0.929,0.948,0.945,0.938,0.969,0.973,0.947,1.000,NaN,NaN
PLATINUM,NaN,0.0,0.000,0.000,0.000,0.000,0.024,0.016,0.065,0.101,...,0.951,0.982,0.968,0.979,0.968,1.000,0.933,NaN,1.000,NaN
EMERALD,NaN,NaN,NaN,0.000,0.053,0.040,0.023,0.050,0.033,0.065,...,0.981,0.934,0.964,0.953,0.972,0.950,1.000,NaN,NaN,NaN
DIAMOND,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.023,0.034,0.034,...,0.961,0.942,0.978,1.000,0.917,1.000,NaN,NaN,1.000,NaN
MASTER,NaN,NaN,NaN,NaN,0.000,0.000,0.032,0.027,0.047,0.029,...,0.926,0.981,1.000,1.000,0.938,1.000,1.000,0.938,NaN,NaN
GRANDMASTER,NaN,NaN,NaN,0.000,NaN,0.000,0.077,0.000,0.074,0.000,...,0.846,0.966,0.944,0.889,1.000,1.000,NaN,NaN,NaN,NaN



=== Tier × dragonDiff 승률 비교 ===


...

diff_bin,-3.0,-2.0,-1.0,-0.0,1.0,2.0,3.0
tier,,,,,,,
IRON,0.076,0.141,0.267,0.509,0.744,0.887,0.943
BRONZE,0.072,0.141,0.274,0.518,0.744,0.844,0.928
SILVER,0.083,0.143,0.293,0.491,0.723,0.854,0.925
GOLD,0.069,0.148,0.297,0.493,0.714,0.877,0.932
PLATINUM,0.095,0.150,0.263,0.493,0.742,0.853,0.947
EMERALD,0.054,0.136,0.257,0.489,0.700,0.839,0.916
DIAMOND,0.091,0.155,0.286,0.491,0.702,0.845,0.910
MASTER,0.087,0.151,0.285,0.489,0.707,0.861,0.931
GRANDMASTER,0.073,0.146,0.278,0.510,0.714,0.851,0.923



=== Tier × innerTowerDiff 승률 비교 ===


diff_bin,-3.0,-2.0,-1.0,-0.0,1.0,2.0,3.0
tier,,,,,,,
IRON,0.096,0.169,0.269,0.519,0.729,0.824,0.916
BRONZE,0.092,0.173,0.289,0.523,0.720,0.796,0.904
SILVER,0.101,0.174,0.276,0.512,0.710,0.779,0.897
GOLD,0.093,0.171,0.294,0.514,0.712,0.804,0.892
PLATINUM,0.098,0.170,0.297,0.498,0.723,0.830,0.921
EMERALD,0.083,0.147,0.255,0.488,0.689,0.812,0.923
DIAMOND,0.105,0.198,0.282,0.492,0.696,0.828,0.918
MASTER,0.090,0.178,0.250,0.508,0.712,0.842,0.926
GRANDMASTER,0.066,0.139,0.260,0.518,0.743,0.813,0.874



=== Tier × baseTowerDiff 승률 비교 ===


diff_bin,-3.0,-2.0,-1.0,-0.0,1.0,2.0,3.0
tier,,,,,,,
IRON,0.132,0.167,0.238,0.511,0.778,0.883,0.946
BRONZE,0.110,0.144,0.244,0.520,0.772,0.842,0.904
SILVER,0.189,0.126,0.219,0.511,0.749,0.830,0.824
GOLD,0.134,0.169,0.219,0.512,0.761,0.870,0.901
PLATINUM,0.087,0.141,0.224,0.500,0.789,0.873,0.921
EMERALD,0.082,0.120,0.183,0.487,0.750,0.830,0.923
DIAMOND,0.136,0.106,0.219,0.497,0.778,0.815,0.944
MASTER,0.156,0.124,0.207,0.510,0.792,0.851,0.900
GRANDMASTER,0.261,0.086,0.207,0.516,0.808,0.850,0.794


In [37]:
import pandas as pd
import os

# ==================================================
# 1. CSV 로드
# ==================================================
final_df = pd.read_csv(
    "../../data/analysis/tier/tier_metric_winrate.csv",
    encoding="utf-8-sig"
)

# ==================================================
# 2. 티어 순서 정의
# ==================================================
TIER_ORDER = [
    "IRON", "BRONZE", "SILVER", "GOLD",
    "PLATINUM", "EMERALD", "DIAMOND",
    "MASTER", "GRANDMASTER", "CHALLENGER"
]

final_df["tier"] = pd.Categorical(
    final_df["tier"],
    categories=TIER_ORDER,
    ordered=True
)

# ==================================================
# 3. grubDiff 범위 강제 제한 (-3 ~ +3)
# ==================================================
final_df = final_df[
    ~(
        (final_df["metric"] == "grubDiff") &
        ((final_df["diff_bin"] < -3) | (final_df["diff_bin"] > 3))
    )
]

# ==================================================
# 4. metric 목록
# ==================================================
metrics = final_df["metric"].unique()

# ==================================================
# 5. 저장 디렉터리 생성
# ==================================================
OUTPUT_DIR = "../../data/analysis/tier/metrics"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ==================================================
# 6. metric별 Pivot 테이블 생성 + 저장
# ==================================================
for metric in metrics:
    metric_df = final_df[final_df["metric"] == metric].copy()

    if metric_df.empty:
        continue

    pivot = (
        metric_df
        .pivot(
            index="tier",        # 행: 티어
            columns="diff_bin",  # 열: diff 값
            values="win_rate"    # 값: 승률
        )
        .sort_index()           # 티어 순서
        .sort_index(axis=1)     # diff_bin 오름차순
    )

    # ===============================
    # CSV 저장
    # ===============================
    output_path = os.path.join(
        OUTPUT_DIR,
        f"tier_{metric}_winrate_pivot.csv"
    )

    pivot.to_csv(output_path, encoding="utf-8-sig")

    # ===============================
    # 출력
    # ===============================
    print(f"\n=== Tier × {metric} 승률 비교 ===")
    print(f"[SAVE] {output_path}")
    display(pivot.round(3))



=== Tier × goldDiff 승률 비교 ===
[SAVE] ../../data/analysis/tier/metrics\tier_goldDiff_winrate_pivot.csv


diff_bin,-16000.0,-15000.0,-14500.0,-14000.0,-13500.0,-13000.0,-12500.0,-12000.0,-11500.0,-11000.0,...,10000.0,10500.0,11000.0,11500.0,12000.0,12500.0,13000.0,13500.0,14000.0,14500.0
tier,,,,,,,,,,,,,,,,,,,,,
IRON,0.0,NaN,NaN,0.000,0.000,0.000,0.000,0.025,0.000,0.058,...,0.978,1.000,0.978,1.000,0.955,1.000,1.0,1.0,1.0,NaN
BRONZE,NaN,0.0,0.0,0.000,0.000,0.000,0.042,0.024,0.038,0.000,...,1.000,0.947,1.000,0.980,0.950,1.000,1.0,1.0,NaN,NaN
SILVER,NaN,NaN,0.0,0.000,0.000,0.000,0.000,0.000,0.020,0.016,...,0.978,0.987,1.000,1.000,1.000,0.962,1.0,1.0,1.0,NaN
GOLD,NaN,0.0,0.0,0.062,0.000,0.077,0.000,0.000,0.023,0.000,...,0.988,0.985,1.000,1.000,1.000,1.000,1.0,1.0,1.0,1.0
PLATINUM,NaN,0.0,0.0,0.000,0.056,0.056,0.000,0.026,0.000,0.014,...,1.000,1.000,1.000,0.977,1.000,1.000,1.0,1.0,1.0,NaN
EMERALD,NaN,NaN,0.0,0.062,0.000,0.038,0.053,0.000,0.017,0.034,...,0.986,1.000,0.977,0.973,1.000,0.957,1.0,NaN,1.0,NaN
DIAMOND,NaN,NaN,0.0,0.000,0.000,0.000,0.000,0.000,0.023,0.000,...,0.984,1.000,0.956,0.971,0.964,1.000,1.0,NaN,NaN,NaN
MASTER,NaN,NaN,0.0,0.000,0.000,0.000,0.031,0.000,0.029,0.000,...,0.973,1.000,1.000,1.000,0.960,1.000,1.0,1.0,NaN,NaN
GRANDMASTER,NaN,NaN,NaN,NaN,0.000,NaN,0.053,0.000,0.000,0.000,...,1.000,0.968,1.000,0.969,1.000,1.000,NaN,1.0,NaN,NaN



=== Tier × totalKillDiff 승률 비교 ===
[SAVE] ../../data/analysis/tier/metrics\tier_totalKillDiff_winrate_pivot.csv


diff_bin,-27.0,-26.0,-25.0,-24.0,-23.0,-22.0,-21.0,-20.0,-19.0,-18.0,...,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0
tier,,,,,,,,,,,,,,,,,,,,,
IRON,0.091,0.0,0.083,0.111,0.156,0.119,0.055,0.099,0.065,0.061,...,0.969,0.965,0.922,0.982,0.974,0.977,1.000,0.957,0.920,1.0
BRONZE,0.000,NaN,0.053,0.048,0.000,0.023,0.022,0.038,0.054,0.042,...,0.957,0.936,0.957,0.960,0.844,0.972,0.955,1.000,0.882,NaN
SILVER,0.100,NaN,0.050,0.000,0.000,0.000,0.036,0.035,0.083,0.067,...,0.897,0.951,1.000,0.981,1.000,0.933,1.000,1.000,1.000,NaN
GOLD,NaN,NaN,NaN,0.000,0.000,0.037,0.000,0.056,0.048,0.039,...,0.929,0.948,0.945,0.938,0.969,0.973,0.947,1.000,NaN,NaN
PLATINUM,NaN,0.0,0.000,0.000,0.000,0.000,0.024,0.016,0.065,0.101,...,0.951,0.982,0.968,0.979,0.968,1.000,0.933,NaN,1.000,NaN
EMERALD,NaN,NaN,NaN,0.000,0.053,0.040,0.023,0.050,0.033,0.065,...,0.981,0.934,0.964,0.953,0.972,0.950,1.000,NaN,NaN,NaN
DIAMOND,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.023,0.034,0.034,...,0.961,0.942,0.978,1.000,0.917,1.000,NaN,NaN,1.000,NaN
MASTER,NaN,NaN,NaN,NaN,0.000,0.000,0.032,0.027,0.047,0.029,...,0.926,0.981,1.000,1.000,0.938,1.000,1.000,0.938,NaN,NaN
GRANDMASTER,NaN,NaN,NaN,0.000,NaN,0.000,0.077,0.000,0.074,0.000,...,0.846,0.966,0.944,0.889,1.000,1.000,NaN,NaN,NaN,NaN



=== Tier × dragonDiff 승률 비교 ===
[SAVE] ../../data/analysis/tier/metrics\tier_dragonDiff_winrate_pivot.csv


...

diff_bin,-3.0,-2.0,-1.0,-0.0,1.0,2.0,3.0
tier,,,,,,,
IRON,0.076,0.141,0.267,0.509,0.744,0.887,0.943
BRONZE,0.072,0.141,0.274,0.518,0.744,0.844,0.928
SILVER,0.083,0.143,0.293,0.491,0.723,0.854,0.925
GOLD,0.069,0.148,0.297,0.493,0.714,0.877,0.932
PLATINUM,0.095,0.150,0.263,0.493,0.742,0.853,0.947
EMERALD,0.054,0.136,0.257,0.489,0.700,0.839,0.916
DIAMOND,0.091,0.155,0.286,0.491,0.702,0.845,0.910
MASTER,0.087,0.151,0.285,0.489,0.707,0.861,0.931
GRANDMASTER,0.073,0.146,0.278,0.510,0.714,0.851,0.923



=== Tier × innerTowerDiff 승률 비교 ===
[SAVE] ../../data/analysis/tier/metrics\tier_innerTowerDiff_winrate_pivot.csv


diff_bin,-3.0,-2.0,-1.0,-0.0,1.0,2.0,3.0
tier,,,,,,,
IRON,0.096,0.169,0.269,0.519,0.729,0.824,0.916
BRONZE,0.092,0.173,0.289,0.523,0.720,0.796,0.904
SILVER,0.101,0.174,0.276,0.512,0.710,0.779,0.897
GOLD,0.093,0.171,0.294,0.514,0.712,0.804,0.892
PLATINUM,0.098,0.170,0.297,0.498,0.723,0.830,0.921
EMERALD,0.083,0.147,0.255,0.488,0.689,0.812,0.923
DIAMOND,0.105,0.198,0.282,0.492,0.696,0.828,0.918
MASTER,0.090,0.178,0.250,0.508,0.712,0.842,0.926
GRANDMASTER,0.066,0.139,0.260,0.518,0.743,0.813,0.874



=== Tier × baseTowerDiff 승률 비교 ===
[SAVE] ../../data/analysis/tier/metrics\tier_baseTowerDiff_winrate_pivot.csv


diff_bin,-3.0,-2.0,-1.0,-0.0,1.0,2.0,3.0
tier,,,,,,,
IRON,0.132,0.167,0.238,0.511,0.778,0.883,0.946
BRONZE,0.110,0.144,0.244,0.520,0.772,0.842,0.904
SILVER,0.189,0.126,0.219,0.511,0.749,0.830,0.824
GOLD,0.134,0.169,0.219,0.512,0.761,0.870,0.901
PLATINUM,0.087,0.141,0.224,0.500,0.789,0.873,0.921
EMERALD,0.082,0.120,0.183,0.487,0.750,0.830,0.923
DIAMOND,0.136,0.106,0.219,0.497,0.778,0.815,0.944
MASTER,0.156,0.124,0.207,0.510,0.792,0.851,0.900
GRANDMASTER,0.261,0.086,0.207,0.516,0.808,0.850,0.794
